<a href="https://colab.research.google.com/github/drfperez/openair/blob/main/OpenAir1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
###############################################################
# ANÀLISI DE LA CONTAMINACIÓ DE L'AIRE A MARTORELL (1991-2025)
# NIVELL: 4t ESO -DIGITALITZACiÓ- BATXILLERAT – PROGRAMACIÓ
#
# OBJECTIUS DEL CODI:
# - Importar dades de contaminació atmosfèrica
# - Preparar les dates correctament
# - Analitzar variacions temporals dels contaminants
# - Visualitzar episodis de contaminació
# - Detectar superacions de la normativa europea
#
# NORMATIVA AMBIENTAL UTILITZADA:
# Directiva 2008/50/CE de qualitat de l'aire de la Unió Europea
#
# LÍMITS PRINCIPALS:
# NO2:
#   - 200 µg/m³ límit horari
#   - 40 µg/m³ límit anual
#
# PM10:
#   - 50 µg/m³ límit diari (màxim 35 dies/any)
#   - 40 µg/m³ límit anual
###############################################################

##############################
# 1. CARREGAR LLIBRERIES
##############################

# openair és una llibreria especialitzada en anàlisi de qualitat de l'aire
library(openair)

##############################
# 2. IMPORTAR DADES
##############################

# Llegim el fitxer de dades ambientals
# Cada columna representa un contaminant atmosfèric
ciutatwide <- read.csv("processed_data_wide.csv")

##############################
# 3. PREPARAR LA COLUMNA DE DATA
##############################

# Convertim la columna "date" de text a format temporal real
# Això permet fer anàlisis per hores, dies, mesos i anys
ciutatwide$date <- as.POSIXct(
  ciutatwide$date,
  "%Y-%m-%d %H:%M:%S",
  tz = "Europe/Madrid"
)

##############################
# 4. VISUALITZACIÓ GENERAL DE DADES
##############################

# Mostra la taula de dades (opcional en entorn gràfic)
View(ciutatwide)

##############################
# 5. VARIACIÓ TEMPORAL GLOBAL
##############################

# Analitza com varien els contaminants:
# - al llarg del dia
# - durant la setmana
# - durant l'any
# - al llarg de dècades

timeVariation(
  ciutatwide,
  pollutant = c(
    "co","h2s","hcnm","hct",
    "no","no2","nox","o3",
    "pm10","pm2.5","so2"
  ),
  main = "Contaminació de l'aire a Martorell (1991-2025)"
)

##############################
# 6. FILTRAR DADES DE L'ANY 2025
##############################

# Seleccionem només registres de l'any 2025
dades_2025 <- subset(
  ciutatwide,
  format(date, "%Y") == "2025"
)

##############################
# 7. VARIACIÓ TEMPORAL 2025
##############################

# Analitza els principals contaminants relacionats amb el trànsit
timeVariation(
  dades_2025,
  pollutant = c("no","no2","nox","o3","pm10","pm2.5"),
  main = "Contaminació de l'aire a Martorell (2025)"
)

##############################
# 8. CALENDAR PLOT – PM10
##############################

# Representa cada dia com una casella de calendari
# Permet detectar episodis puntuals de contaminació
calendarPlot(
  dades_2025,
  pollutant = "pm10",
  year = 2025,
  limits = c(0,80),
  main = "PM10 – Martorell (2025)"
)

##############################
# 9. SUPERACIONS DIÀRIES PM10
##############################

# Límit legal europeu PM10 diari:
# 50 µg/m³
calendarPlot(
  dades_2025,
  pollutant = "pm10",
  year = 2025,
  exceedance = 50,
  main = "PM10 – Superacions del límit diari (50 µg/m³)"
)

##############################
# 10. CALENDAR AMB RANGS DE QUALITAT
##############################

calendarPlot(
  dades_2025,
  pollutant = "pm10",
  year = 2025,
  limits = c(0,25,50,150),
  cols = c("green","orange","red"),
  main = "PM10 – Verd <25 | Taronja 25-50 | Vermell >50"
)

##############################
# 11. TENDÈNCIA HISTÒRICA NO2
##############################

# Permet veure si la contaminació augmenta o disminueix
trendLevel(
  ciutatwide,
  pollutant = "no2",
  main = "Tendència anual de NO2 (1991-2025)"
)

##############################
# 12. CÀLCUL DE SUPERACIONS LEGALS
##############################

### NO2 – LÍMIT HORARI
# 200 µg/m³
no2_hourly_exceed <- sum(
  ciutatwide$no2 > 200,
  na.rm = TRUE
)

cat("Hores amb NO2 > 200 µg/m³:", no2_hourly_exceed, "\n")

### NO2 – LÍMIT ANUAL
# 40 µg/m³
no2_annual <- timeAverage(
  ciutatwide,
  avg.time = "year",
  pollutant = "no2"
)

no2_annual_exceed <- sum(
  no2_annual$no2 > 40,
  na.rm = TRUE
)

cat("Anys amb NO2 > 40 µg/m³:", no2_annual_exceed, "\n")

##############################
# 13. PM10 – MITJANES DIÀRIES
##############################

pm10_daily <- timeAverage(
  ciutatwide,
  avg.time = "day",
  pollutant = "pm10"
)

pm10_daily_exceed <- sum(
  pm10_daily$pm10 > 50,
  na.rm = TRUE
)

cat("Dies amb PM10 > 50 µg/m³:", pm10_daily_exceed, "\n")

##############################
# 14. PM10 – MITJANES ANUALS
##############################

pm10_annual <- timeAverage(
  ciutatwide,
  avg.time = "year",
  pollutant = "pm10"
)

pm10_annual_exceed <- sum(
  pm10_annual$pm10 > 40,
  na.rm = TRUE
)

cat("Anys amb PM10 > 40 µg/m³:", pm10_annual_exceed, "\n")

##############################
# 15. IDENTIFICAR EPISODIS CONCRETS
##############################

# Dies concrets amb contaminació elevada de PM10
pm10_exceed_days <- subset(
  pm10_daily,
  pm10 > 50
)

pm10_exceed_days[,c("date","pm10")]

# Hores concretes amb NO2 elevat
no2_exceed_hours <- subset(
  ciutatwide,
  no2 > 200
)

no2_exceed_hours[,c("date","no2")]

# Anys amb excés anual de NO2
no2_annual_exceed <- subset(
  no2_annual,
  no2 > 40
)

no2_annual_exceed[,c("date","no2")]


##############################
# 15b. TAULA COMPLETA DE SUPERACIONS (1991-2021)
##############################

# Límits legals segons Directiva UE 2008/50/CE
limits <- list(
  no2_hour = 200,    # µg/m³
  no2_year = 40,     # µg/m³
  pm10_day = 50,     # µg/m³
  pm10_year = 40,    # µg/m³
  pm25_day = 25,     # µg/m³ nou límit UE 2024, s'integra en anual
  pm25_year = 10,    # µg/m³
  o3_hour8 = 120,    # µg/m³ valor objectiu UE 2024
  o3_warm = 60,      # µg/m³ pic 6 mesos OMS 2021
  so2_hour = 350,    # µg/m³
  so2_day = 125,     # µg/m³
  so2_year = 20,     # µg/m³
  co_hour8 = 10,     # mg/m³
  co_day = 10,       # mg/m³, 24h OMS
  benzene_year = 5,  # µg/m³ històric UE
  pb_year = 0.5      # µg/m³
)

# Filtrar dades 1991-2021
dades_period <- subset(
  ciutatwide,
  format(date, "%Y") >= "1991" & format(date, "%Y") <= "2021"
)

# Inicialitzem taula buida
superacions <- data.frame(
  contaminant = character(),
  tipus_limit = character(),
  data = as.POSIXct(character()),
  valor_mesurat = numeric(),
  stringsAsFactors = FALSE
)

##############################
# FUNCIONS AUXILIARS
##############################

# Afegir superacions horàries
add_hourly_exceed <- function(contaminant, limit){
  subset_data <- subset(dades_period, dades_period[[contaminant]] > limit)
  if(nrow(subset_data) > 0){
    tmp <- data.frame(
      contaminant = contaminant,
      tipus_limit = "hora",
      data = subset_data$date,
      valor_mesurat = subset_data[[contaminant]]
    )
    return(tmp)
  } else {
    return(NULL)
  }
}

# Afegir superacions diàries
add_daily_exceed <- function(contaminant, limit){
  daily_avg <- timeAverage(dades_period, avg.time = "day", pollutant = contaminant)
  subset_data <- subset(daily_avg, daily_avg[[contaminant]] > limit)
  if(nrow(subset_data) > 0){
    tmp <- data.frame(
      contaminant = contaminant,
      tipus_limit = "dia",
      data = subset_data$date,
      valor_mesurat = subset_data[[contaminant]]
    )
    return(tmp)
  } else {
    return(NULL)
  }
}

# Afegir superacions anuals
add_yearly_exceed <- function(contaminant, limit){
  yearly_avg <- timeAverage(dades_period, avg.time = "year", pollutant = contaminant)
  subset_data <- subset(yearly_avg, yearly_avg[[contaminant]] > limit)
  if(nrow(subset_data) > 0){
    tmp <- data.frame(
      contaminant = contaminant,
      tipus_limit = "any",
      data = subset_data$date,
      valor_mesurat = subset_data[[contaminant]]
    )
    return(tmp)
  } else {
    return(NULL)
  }
}

##############################
# 15b. SUPERACIONS PER CADA CONTAMINANT
##############################

# NO2
superacions <- rbind(superacions, add_hourly_exceed("no2", limits$no2_hour))
superacions <- rbind(superacions, add_yearly_exceed("no2", limits$no2_year))

# PM10
superacions <- rbind(superacions, add_daily_exceed("pm10", limits$pm10_day))
superacions <- rbind(superacions, add_yearly_exceed("pm10", limits$pm10_year))

# PM2.5
superacions <- rbind(superacions, add_daily_exceed("pm2.5", limits$pm25_day))
superacions <- rbind(superacions, add_yearly_exceed("pm2.5", limits$pm25_year))

# SO2
superacions <- rbind(superacions, add_hourly_exceed("so2", limits$so2_hour))
superacions <- rbind(superacions, add_daily_exceed("so2", limits$so2_day))
superacions <- rbind(superacions, add_yearly_exceed("so2", limits$so2_year))

# CO
superacions <- rbind(superacions, add_hourly_exceed("co", limits$co_hour8))
superacions <- rbind(superacions, add_daily_exceed("co", limits$co_day))

# O3 (8h)
superacions <- rbind(superacions, add_hourly_exceed("o3", limits$o3_hour8))

# O3 temporada càlida (OMS 2021)
# Assumim columna "o3_warm" amb mitjana de 6 mesos calents
if("o3_warm" %in% names(dades_period)){
  superacions <- rbind(superacions, add_yearly_exceed("o3_warm", limits$o3_warm))
}

# Benzè anual
superacions <- rbind(superacions, add_yearly_exceed("benzene", limits$benzene_year))

# Plom anual
superacions <- rbind(superacions, add_yearly_exceed("pb", limits$pb_year))

##############################
# ORDENAR TAULA
##############################
superacions <- superacions[order(superacions$contaminant,
                                 superacions$tipus_limit,
                                 superacions$data),]

# Mostrar taula completa
View(superacions)

# Exportar a CSV
write.csv(superacions, "superacions_completes_1991_2021.csv", row.names = FALSE)

cat("Taula completa de superacions generada amb", nrow(superacions), "registre(s)\n")


#####################################################
# FINAL DEL SCRIPT
###############################################################

In [ ]:
##############################
# 15b (ampliat). RESUM ANUAL I TAULA COMPLETA DE SUPERACIONS (1991-2021)
##############################

library(dplyr) # per agrupar/operacions; si no està instal·lat: install.packages("dplyr")

# REUTILITZEM 'dades_period' com a subconjunt 1991-2021 (creat abans)
# Si no existeix, el creem ara:
if(!exists("dades_period")){
  dades_period <- subset(ciutatwide, format(date, "%Y") >= "1991" & format(date, "%Y") <= "2021")
}

# Límits (ajusta segons prefereixis)
limits <- list(
  no2_hour = 200,    # µg/m³
  no2_year = 40,     # µg/m³
  pm10_day = 50,     # µg/m³
  pm10_year = 40,    # µg/m³
  pm25_day = 25,     # µg/m³ (valors històrics/UE/2024 segons context)
  pm25_year = 10,    # µg/m³
  so2_hour = 350,    # µg/m³
  so2_day = 125,     # µg/m³
  so2_year = 20,     # µg/m³
  co_hour8 = 10,     # mg/m³ (8h)
  co_day = 10,       # mg/m³ (24h, OMS)
  o3_hour8 = 120,    # µg/m³ (valor objectiu UE)
  benzene_year = 5,  # µg/m³ (UE històric)
  pb_year = 0.5      # µg/m³
)

# Contaminants a analitzar (inclou els noms de les columnes del teu dataframe)
contaminants <- c("no2","pm10","pm2.5","so2","co","o3","benzene","pb")

years <- 1991:2021

# TAULA DETAILED: ja creada prèviament com 'superacions' en l'anterior 15b,
# però la regenerem aquí de forma robusta per assegurar coherència
superacions <- data.frame(
  contaminant = character(),
  tipus_limit = character(),   # "hora", "dia", "any"
  data = as.POSIXct(character()),
  valor_mesurat = numeric(),
  stringsAsFactors = FALSE
)

# Funció per comprovar existència de columna
col_exists <- function(df, col) {
  return(col %in% names(df))
}

# Recorrem contaminants i anys
for(cont in contaminants){
  if(!col_exists(dades_period, cont)){
    message(sprintf("Avis: columna '%s' no trobada a dades_period → es salta.", cont))
    next
  }
  for(y in years){
    start <- as.POSIXct(sprintf("%d-01-01 00:00:00", y), tz = "Europe/Madrid")
    end   <- as.POSIXct(sprintf("%d-12-31 23:59:59", y), tz = "Europe/Madrid")
    year_data <- subset(dades_period, date >= start & date <= end)
    # 1) Horari (si existeix límit horari per a aquest contaminant)
    # mapeig de claus de limits segons pollutant
    # definim claus possibles
    hour_key <- switch(cont,
                       "no2" = "no2_hour",
                       "so2" = "so2_hour",
                       "co"  = "co_hour8",
                       "o3"  = "o3_hour8",
                       NULL)
    day_key <- switch(cont,
                      "pm10" = "pm10_day",
                      "pm2.5" = "pm25_day",
                      "so2"   = "so2_day",
                      "co"    = "co_day",
                      NULL)
    year_key <- switch(cont,
                       "no2" = "no2_year",
                       "pm10" = "pm10_year",
                       "pm2.5" = "pm25_year",
                       "so2" = "so2_year",
                       "benzene" = "benzene_year",
                       "pb" = "pb_year",
                       NULL)

    # Horari
    if(!is.null(hour_key) && !is.null(limits[[hour_key]]) && nrow(year_data) > 0){
      idx <- which(!is.na(year_data[[cont]]) & year_data[[cont]] > limits[[hour_key]])
      if(length(idx) > 0){
        tmp <- data.frame(
          contaminant = cont,
          tipus_limit = "hora",
          data = year_data$date[idx],
          valor_mesurat = year_data[[cont]][idx],
          stringsAsFactors = FALSE
        )
        superacions <- bind_rows(superacions, tmp)
      }
    }

    # Diari
    if(!is.null(day_key) && !is.null(limits[[day_key]]) && nrow(year_data) > 0){
      daily <- timeAverage(year_data, avg.time = "day", pollutant = cont)
      if(nrow(daily) > 0){
        idxd <- which(!is.na(daily[[cont]]) & daily[[cont]] > limits[[day_key]])
        if(length(idxd) > 0){
          tmp <- data.frame(
            contaminant = cont,
            tipus_limit = "dia",
            data = daily$date[idxd],
            valor_mesurat = daily[[cont]][idxd],
            stringsAsFactors = FALSE
          )
          superacions <- bind_rows(superacions, tmp)
        }
      }
    }

    # Any
    if(!is.null(year_key) && !is.null(limits[[year_key]]) && nrow(year_data) > 0){
      yearly <- timeAverage(year_data, avg.time = "year", pollutant = cont)
      if(nrow(yearly) > 0 && !is.na(yearly[[cont]][1])){
        year_val <- yearly[[cont]][1]
        if(!is.na(year_val) && year_val > limits[[year_key]]){
          tmp <- data.frame(
            contaminant = cont,
            tipus_limit = "any",
            data = as.POSIXct(sprintf("%d-01-01 00:00:00", y), tz = "Europe/Madrid"),
            valor_mesurat = year_val,
            stringsAsFactors = FALSE
          )
          superacions <- bind_rows(superacions, tmp)
        }
      }
    }
  } # fi loop anys
} # fi loop contaminants

# Ordenar la taula detallada
superacions <- superacions %>%
  arrange(contaminant, tipus_limit, data)

# Guardar la taula detallada
write.csv(superacions, "superacions_completes_1991_2021.csv", row.names = FALSE)
message(sprintf("Taula detallada guardada: %d registres -> superacions_completes_1991_2021.csv", nrow(superacions)))

# ---------- RESUM ANUAL (per contaminant i any) ----------
# Per cada contaminant i any: nombre d'hores superades, nombre de dies superats, valor anual i indicador any superat (0/1)

res_list <- list()
for(cont in contaminants){
  if(!col_exists(dades_period, cont)) next
  for(y in years){
    start <- as.POSIXct(sprintf("%d-01-01 00:00:00", y), tz = "Europe/Madrid")
    end   <- as.POSIXct(sprintf("%d-12-31 23:59:59", y), tz = "Europe/Madrid")
    year_data <- subset(dades_period, date >= start & date <= end)

    hour_key <- switch(cont,
                       "no2" = "no2_hour",
                       "so2" = "so2_hour",
                       "co"  = "co_hour8",
                       "o3"  = "o3_hour8",
                       NULL)
    day_key <- switch(cont,
                      "pm10" = "pm10_day",
                      "pm2.5" = "pm25_day",
                      "so2"   = "so2_day",
                      "co"    = "co_day",
                      NULL)
    year_key <- switch(cont,
                       "no2" = "no2_year",
                       "pm10" = "pm10_year",
                       "pm2.5" = "pm25_year",
                       "so2" = "so2_year",
                       "benzene" = "benzene_year",
                       "pb" = "pb_year",
                       NULL)

    hours_exceeded <- NA_integer_
    days_exceeded  <- NA_integer_
    year_mean_val  <- NA_real_
    year_exceeded  <- NA_integer_

    # hores
    if(!is.null(hour_key) && !is.null(limits[[hour_key]]) && nrow(year_data) > 0){
      hours_exceeded <- sum(!is.na(year_data[[cont]]) & year_data[[cont]] > limits[[hour_key]])
    } else {
      hours_exceeded <- NA_integer_
    }

    # dies
    if(!is.null(day_key) && !is.null(limits[[day_key]]) && nrow(year_data) > 0){
      daily <- timeAverage(year_data, avg.time = "day", pollutant = cont)
      if(nrow(daily) > 0){
        days_exceeded <- sum(!is.na(daily[[cont]]) & daily[[cont]] > limits[[day_key]])
      } else {
        days_exceeded <- 0
      }
    } else {
      days_exceeded <- NA_integer_
    }

    # any
    if(!is.null(year_key) && !is.null(limits[[year_key]]) && nrow(year_data) > 0){
      yearly <- timeAverage(year_data, avg.time = "year", pollutant = cont)
      if(nrow(yearly) > 0){
        year_mean_val <- yearly[[cont]][1]
        year_exceeded <- as.integer(!is.na(year_mean_val) && year_mean_val > limits[[year_key]])
      } else {
        year_mean_val <- NA_real_
        year_exceeded <- NA_integer_
      }
    } else {
      year_mean_val <- NA_real_
      year_exceeded <- NA_integer_
    }

    res_list[[length(res_list) + 1]] <- data.frame(
      contaminant = cont,
      year = y,
      hours_exceeded = hours_exceeded,
      days_exceeded = days_exceeded,
      year_mean = year_mean_val,
      year_exceeded = year_exceeded,
      stringsAsFactors = FALSE
    )
  }
}

resum_anual <- bind_rows(res_list) %>%
  arrange(contaminant, year)

# Guardar resum anual
write.csv(resum_anual, "resum_anual_superacions_1991_2021.csv", row.names = FALSE)
message(sprintf("Resum anual guardat: %d files -> resum_anual_superacions_1991_2021.csv", nrow(resum_anual)))

# Mostrar les primeres files per revisar
print(head(resum_anual, 20))

# Opcional: resum agregat per contaminant (total període)
res_aggregat <- resum_anual %>%
  group_by(contaminant) %>%
  summarise(
    total_hours_exceeded = sum(replace_na(hours_exceeded, 0)),
    total_days_exceeded  = sum(replace_na(days_exceeded, 0)),
    years_exceeded_count = sum(replace_na(year_exceeded, 0)),
    years_with_data = sum(!is.na(year_mean))
  ) %>%
  arrange(desc(total_days_exceeded), desc(total_hours_exceeded))

print(res_aggregat)

In [ ]:
##############################
# 15b.1 TAULA RESUM ANUAL (FORMAT WIDE)
##############################

library(tidyr)
library(dplyr)

# Partim de la taula "resum_anual" creada abans
# Columnes:
# contaminant | year | hours_exceeded | days_exceeded | year_mean | year_exceeded

# Convertim a format wide
resum_wide <- resum_anual %>%
  pivot_wider(
    id_cols = year,
    names_from = contaminant,
    values_from = c(hours_exceeded, days_exceeded, year_exceeded),
    names_sep = "_"
  ) %>%
  arrange(year)

# Mostrar taula
View(resum_wide)

# Exportar a CSV
write.csv(
  resum_wide,
  "resum_anual_superacions_WIDE_1991_2021.csv",
  row.names = FALSE
)

cat("Taula resum anual WIDE creada correctament\n")

In [ ]:
##############################
# 15b.2 GRÀFIC: TOTAL DE DIES SUPERATS PER CONTAMINANT
##############################

resum_dies <- resum_anual %>%
  group_by(contaminant) %>%
  summarise(
    total_dies_superats = sum(days_exceeded, na.rm = TRUE)
  ) %>%
  arrange(desc(total_dies_superats))

barplot(
  resum_dies$total_dies_superats,
  names.arg = resum_dies$contaminant,
  las = 2,
  main = "Total de dies amb superació del límit (1991–2021)",
  ylab = "Nombre de dies",
  xlab = "Contaminant"
)

In [ ]:
##############################
# 15b.3 GRÀFIC: ANYS AMB SUPERACIÓ ANUAL
##############################

resum_anys <- resum_anual %>%
  group_by(contaminant) %>%
  summarise(
    anys_incompliment = sum(year_exceeded, na.rm = TRUE)
  ) %>%
  arrange(desc(anys_incompliment))

barplot(
  resum_anys$anys_incompliment,
  names.arg = resum_anys$contaminant,
  las = 2,
  main = "Anys que NO compleixen el límit anual (1991–2021)",
  ylab = "Nombre d'anys",
  xlab = "Contaminant"
)

In [ ]:
##############################
# 15b.4 EVOLUCIÓ ANUAL (EXEMPLE PM10)
##############################

pm10_years <- resum_anual %>%
  filter(contaminant == "pm10")

plot(
  pm10_years$year,
  pm10_years$days_exceeded,
  type = "b",
  main = "Evolució anual dels dies amb PM10 > límit",
  xlab = "Any",
  ylab = "Dies amb superació"
)

In [ ]:
##############################
# 16.1 CONCLUSIONS GLOBALS PER CONTAMINANT
##############################

conclusions_contaminant <- resum_anual %>%
  group_by(contaminant) %>%
  summarise(
    anys_analitzats = n(),
    anys_incompliment = sum(year_exceeded, na.rm = TRUE),
    percent_incompliment = round(100 * anys_incompliment / anys_analitzats, 1),
    total_dies_superats = sum(days_exceeded, na.rm = TRUE),
    total_hores_superades = sum(hours_exceeded, na.rm = TRUE)
  )

# Generar text automàtic
for(i in 1:nrow(conclusions_contaminant)){
  cat(
    "➡ El contaminant", conclusions_contaminant$contaminant[i],
    "ha incomplert el límit anual en",
    conclusions_contaminant$anys_incompliment[i], "dels",
    conclusions_contaminant$anys_analitzats[i], "anys analitzats (",
    conclusions_contaminant$percent_incompliment[i], "% del període ).\n",
    "   - Total de dies amb superació: ",
    conclusions_contaminant$total_dies_superats[i], "\n",
    "   - Total d’hores amb superació: ",
    conclusions_contaminant$total_hores_superades[i], "\n\n"
  )
}

In [ ]:
##############################
# 16.2 CONTAMINANT MÉS PROBLEMÀTIC
##############################

top_contaminant <- conclusions_contaminant %>%
  arrange(desc(anys_incompliment)) %>%
  slice(1)

cat(
  "⚠ El contaminant més problemàtic durant el període 1991–2021 ha estat",
  top_contaminant$contaminant,
  "amb incompliment del límit anual en",
  top_contaminant$anys_incompliment,
  "anys.\n"
)

In [ ]:
##############################
# 16.3 ANYS MÉS CRÍTICS
##############################

anys_critics <- resum_anual %>%
  group_by(year) %>%
  summarise(
    contaminants_incomplint = sum(year_exceeded, na.rm = TRUE)
  ) %>%
  arrange(desc(contaminants_incomplint))

View(anys_critics)

cat(
  "📅 L'any més crític del període ha estat",
  anys_critics$year[1],
  "amb",
  anys_critics$contaminants_incomplint[1],
  "contaminants superant el límit anual.\n"
)

In [ ]:
##############################
# 16.4 EVOLUCIÓ TEMPORAL
##############################

mid_year <- median(resum_anual$year)

evolucio <- resum_anual %>%
  mutate(period = ifelse(year <= mid_year, "Antic", "Recent")) %>%
  group_by(contaminant, period) %>%
  summarise(
    mitjana_dies = mean(days_exceeded, na.rm = TRUE),
    .groups = "drop"
  ) %>%
  pivot_wider(
    names_from = period,
    values_from = mitjana_dies
  ) %>%
  mutate(
    tendencia = ifelse(Recent < Antic, "Millora", "Empitjora")
  )

View(evolucio)

In [ ]:
##############################
# 16.5 CONCLUSIÓ FINAL AUTOMÀTICA
##############################

cat(
  "📘 CONCLUSIÓ GENERAL:\n",
  "L'anàlisi de la contaminació atmosfèrica a Martorell durant el període 1991–2021\n",
  "mostra que la qualitat de l'aire ha presentat incompliments recurrents dels\n",
  "límits legals europeus, especialment per contaminants associats al trànsit i\n",
  "a activitats industrials.\n\n",
  "Tot i que en els darrers anys s'observa una tendència general a la millora,\n",
  "alguns contaminants continuen superant els valors recomanats, fet que indica\n",
  "la necessitat de mantenir i reforçar les polítiques de reducció d'emissions.\n"
)

In [ ]:
##############################
# 17.1 LLIBRERIES PER WORD
##############################

install.packages("officer")   # només la primera vegada
install.packages("flextable") # opcional, per taules boniques

library(officer)
library(flextable)

In [ ]:
##############################
# 17.2 TEXT DE CONCLUSIONS AUTOMÀTIQUES
##############################

text_contaminants <- ""

for(i in 1:nrow(conclusions_contaminant)){
  text_contaminants <- paste0(
    text_contaminants,
    "• El contaminant ", conclusions_contaminant$contaminant[i],
    " ha superat el límit anual en ",
    conclusions_contaminant$anys_incompliment[i], " de ",
    conclusions_contaminant$anys_analitzats[i],
    " anys analitzats (",
    conclusions_contaminant$percent_incompliment[i], "% del període). ",
    "En total s'han registrat ",
    conclusions_contaminant$total_dies_superats[i],
    " dies i ",
    conclusions_contaminant$total_hores_superades[i],
    " hores amb superacions dels valors legals.\n"
  )
}

In [ ]:
##############################
# 17.3 CREAR DOCUMENT WORD
##############################

doc <- read_docx()

# Títol
doc <- doc %>%
  body_add_par(
    "Anàlisi de la qualitat de l’aire a Martorell (1991–2021)",
    style = "heading 1"
  )

# Subtítol
doc <- doc %>%
  body_add_par(
    "Conclusions automàtiques basades en dades oficials",
    style = "heading 2"
  )

# Introducció
doc <- doc %>%
  body_add_par(
    "Aquest document presenta les conclusions obtingudes a partir de l’anàlisi "
    %+% "de les dades de contaminació atmosfèrica registrades a Martorell durant "
    %+% "el període 1991–2021. Les conclusions s’han generat automàticament a partir "
    %+% "del compliment o incompliment dels límits establerts per la normativa europea.",
    style = "Normal"
  )

# Conclusions per contaminant
doc <- doc %>%
  body_add_par(
    "Resultats per contaminant",
    style = "heading 2"
  ) %>%
  body_add_par(
    text_contaminants,
    style = "Normal"
  )

# Conclusió general
doc <- doc %>%
  body_add_par(
    "Conclusió general",
    style = "heading 2"
  ) %>%
  body_add_par(
    "L’estudi mostra que, tot i una tendència general de millora en els darrers anys, "
    %+% "durant una part significativa del període analitzat s’han produït superacions "
    %+% "dels límits legals de qualitat de l’aire, especialment en contaminants associats "
    %+% "al trànsit i a l’activitat industrial. Aquest fet posa de manifest la necessitat "
    %+% "de continuar aplicant polítiques ambientals de reducció d’emissions.",
    style = "Normal"
  )

# Guardar document
print(doc, target = "Conclusions_Qualitat_Aire_Martorell_1991_2021.docx")

cat("Document Word creat correctament\n")

In [ ]:
##############################
# 17.4 AFEGIR TAULA AL WORD (OPCIONAL)
##############################

taula_word <- flextable(resum_wide)
taula_word <- autofit(taula_word)

doc <- read_docx("Conclusions_Qualitat_Aire_Martorell_1991_2021.docx")

doc <- doc %>%
  body_add_par("Resum anual de superacions", style = "heading 2") %>%
  body_add_flextable(taula_word)

print(doc, target = "Conclusions_Qualitat_Aire_Martorell_1991_2021.docx")